In [5]:
# test calibration using scipy.optimize

import numpy as np
import scipy.optimize as opt

In [6]:
import unittest

import numpy as np

from openptv_python.calibration import (
    Calibration,
)
from openptv_python.imgcoord import image_coordinates
from openptv_python.orientation import (
    external_calibration,
)
from openptv_python.parameters import ControlPar, OrientPar, VolumePar, read_control_par
from openptv_python.tracking_frame_buf import TargetArray
from openptv_python.trafo import arr_metric_to_pixel

In [7]:
class TestGradientDescent(unittest.TestCase):
    # Based on the C tests in liboptv/check_orientation.c

    def setUp(self):
        control_file_name = "testing_folder/corresp/control.par"
        # self.control = ControlPar(4)
        self.control = read_control_par(control_file_name)

        self.orient_par_file_name = "testing_folder/corresp/orient.par"
        self.orient_par = OrientPar().from_file(self.orient_par_file_name)

        self.cal = Calibration().from_file(
            "testing_folder/calibration/cam1.tif.ori",
            "testing_folder/calibration/cam1.tif.addpar",
        )
        self.orig_cal = Calibration().from_file(
            "testing_folder/calibration/cam1.tif.ori",
            "testing_folder/calibration/cam1.tif.addpar",
        )

        
    def test_external_calibration(self):
        """External calibration using clicked points."""
        ref_pts = np.array(
            [
                [-40.0, -25.0, 8.0],
                [40.0, -15.0, 0.0],
                [40.0, 15.0, 0.0],
                [40.0, 0.0, 8.0],
            ]
        )

        # Fake the image points by back-projection
        targets = arr_metric_to_pixel(
            image_coordinates(ref_pts, self.cal, self.control.mm),
            self.control,
        )

        # Jigg the fake detections to give raw_orient some challenge.
        targets[:, 1] -= 0.1

        self.assertTrue(external_calibration(self.cal, ref_pts, targets, self.control))
        np.testing.assert_array_almost_equal(
            self.cal.get_angles(), self.orig_cal.get_angles(), decimal=3
        )
        np.testing.assert_array_almost_equal(
            self.cal.get_pos(), self.orig_cal.get_pos(), decimal=3
    )

In [8]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_external_calibration (__main__.TestGradientDescent)
External calibration using clicked points. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.120s

OK


Coefficients (beta): [ 1.80534911e-04 -6.77499328e-05 -6.52416361e-04 -1.77426081e-05
 -1.31470856e-07  4.02517087e-06]
Residuals: []
rank: 6
singular_values: None
